In [46]:
import warnings
warnings.filterwarnings("ignore")
import socket,time,math,argparse
import numpy as np
from functions import processCentroids_calib,cameraMatrix_cam1,cameraMatrix_cam2,distCoef_cam1,distCoef_cam2
from cv2 import circle,putText,imshow,waitKey,FONT_HERSHEY_SIMPLEX,destroyAllWindows,triangulatePoints,moveWindow,imwrite
from myLib import isCollinear,isEqual,swapElements,getSignal,getOrder,reshapeCoord,findNearestC
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import CubicSpline
import os


In [47]:
df1,df2 = np.genfromtxt('cam1.csv', delimiter=','),np.genfromtxt('cam2.csv', delimiter=',')
maxTime,maxFrame = max(df1[-1][-2],df2[-1][-2]),max(df1[-1][-1],df2[-1][-1])
print('Maximum captured timestamp: '+str(round(maxTime/1e6,2))+' seconds')
print('#'+str(int(maxFrame))+' frames were captured')

Maximum captured timestamp: 117.93 seconds
#4594 frames were captured


In [48]:
def orderCenterCoord(centerCoord, prevCenterCoord, otherCamOrder = 0):
    centerX, centerY = reshapeCoord(centerCoord)
    # if it is the first image of the sequence
    if len(prevCenterCoord) == 0:  
        order,_ =  getOrder(centerX,centerY)  
        # if it is the second camera
        if otherCamOrder != 0:  
            # if the markers are wrong, swap the extremities
            signal, valid = getSignal(centerX[order[0]], centerX[order[2]],5)
            if signal != otherCamOrder and valid: order = swapElements(order, 0, 2)    
        else:        
            # get base for comparision (first camera only)        
            otherCamOrder,_ = getSignal(centerX[order[0]], centerX[order[2]])
        # sort centers        
        if np.linalg.norm(centerX[order[0]]-centerX[order[1]])>np.linalg.norm(centerX[order[2]]-centerX[order[1]]):
            sortedCenterCoord = np.array((centerCoord[order[0]], centerCoord[order[1]], centerCoord[order[2]]))
        else: sortedCenterCoord = np.array((centerCoord[order[2]], centerCoord[order[1]], centerCoord[order[0]]))
    else:
        # first reshape array of coordinates
        prevCenterX,prevCenterY = reshapeCoord(prevCenterCoord)
        # distance from marker A/B of previous img to center coordiantes of actual img
        distA = np.sqrt(np.power(np.subtract(prevCenterX[0], centerX), 2) + np.power(np.subtract(prevCenterY[0], centerY), 2))
        distB = np.sqrt(np.power(np.subtract(prevCenterX[1], centerX), 2) + np.power(np.subtract(prevCenterY[1], centerY), 2))
        # nearest marker from A is selected and removed as marker B candidate
        nearestA = np.argmin(distA)
        distBCopy = np.delete(distB, nearestA)
        # nearest marker from B is selected and removed as marker C candidate
        nearestBCopy = np.argmin(distBCopy)
        nearestB, = np.where(distB == distBCopy[nearestBCopy])
        distBCopy = np.delete(distBCopy, nearestBCopy)
        # get the missing marker position in array
        nearestC = findNearestC(nearestA, nearestB[0])
        # sort centers        
        sortedCenterCoord = [centerCoord[nearestA], centerCoord[nearestB[0]], centerCoord[nearestC]]
        # check if the ordering is ok
        centerX, centerY = reshapeCoord(sortedCenterCoord)
        prevOrder,axisPrev = getOrder(prevCenterX,prevCenterY)
        order,_ =  getOrder(centerX,centerY,baseAxis=True,axis=axisPrev)
        if (order[1] != 1) or (order[2] != prevOrder[2]):
            if prevOrder[0] == 2: order = swapElements(order,0,2) #if is decreasing, swap                
            sortedCenterCoord = np.array((sortedCenterCoord[order[0]], sortedCenterCoord[order[1]], sortedCenterCoord[order[2]]))
    return sortedCenterCoord, otherCamOrder

In [49]:
os.system('del pics\cam1\*jpg')
invalid,lastValid,tol,swap,certainty,counter,lastValid_i=0,0,0.25,0,False,0,0
intervals,valid,jump=[],[],[]
for i in range(0,df1.shape[0]):
    # get points and check invalidity sequence
    pts,imgNumb = df1[i][0:6].reshape(-1,2),int(df1[i][-1])
    if imgNumb>lastValid+1: invalid = imgNumb-lastValid
    # order markers per proximity and check collinearity
    if isCollinear(*pts) and not isEqual(pts) or np.any(pts<0):     
        if invalid>=10 or not counter: 
            if not certainty and counter: valid[-1]=False
            prev,certainty = [],False
            intervals.append(i)
            valid.append(True)
        else: prev = np.array(df1[lastValid_i][0:6]).reshape(1,-2)
        pts, _ = orderCenterCoord(pts,prev)
        pts = np.array(pts)
    else: 
        jump.append(imgNumb)
        invalid+=1
        continue
    # update internal loop variables
    lastValid,invalid,lastValid_i=imgNumb,0,i
    counter+=1
    df1[i][0:6] = np.copy(pts).reshape(-1,6)
    # check if ABC is in order smaller to largest
    # if 2 consecutive trues, invert all df until interval beginning
    if not certainty:
        for [A,B,C] in pts.reshape([-1, 3, 2]):
            if np.linalg.norm(C-B)/np.linalg.norm(A-B)>(2-tol) and np.linalg.norm(C-B)>20:
                swap += 1
                if swap>5: df1[intervals[-1]:i][0:2],df1[intervals[-1]:i][4:6],swap,certainty = df1[intervals[-1]:i][4:6],df1[intervals[-1]:i][0:2],0,True
            if np.linalg.norm(A-B)/np.linalg.norm(C-B)>(2-tol) and np.linalg.norm(A-B)>20: certainty = True
    #if not i%100: print(i)
intervals.append(i)

for i in range(0,len(intervals)-1):
    if not valid[i]: continue
    beg,end = intervals[i],intervals[i+1]
    ### VERBOSE
    for j in range(beg,end):
        pts,imgNumb,img,k = df1[j][0:6].reshape(-1,2),int(df1[j][-1]),np.ones((540,960,3))*255,0
        if imgNumb in jump: continue
        for k in range(0,3):
            pt = pts.reshape(-1,2)[k]
            center = (int(np.round(pt[0]*16)),int(np.round(pt[1]*16)))
            circle(img,center,10,(255,0,0),5,shift=4)
            putText(img,str(k),(int(center[0]/16)-25, int(center[1]/16)-25),FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2) 
        imwrite('pics/cam1/'+str(imgNumb).zfill(4)+'.jpg',img)
print('Done camera #1')

Done camera #1


In [50]:
os.system('del pics\cam2\*jpg')
invalid,lastValid,tol,swap,certainty,counter,lastValid_i=0,0,0.25,0,False,0,0
intervals,valid,jump=[],[],[]
for i in range(0,df2.shape[0]):
    # get points and check invalidity sequence
    pts,imgNumb = df2[i][0:6].reshape(-1,2),int(df2[i][-1])
    if imgNumb>lastValid+1: invalid = imgNumb-lastValid
    # order markers per proximity and check collinearity
    if isCollinear(*pts) and not isEqual(pts) or np.any(pts<0):     
        if invalid>=10 or not counter: 
            if not certainty and counter: valid[-1]=False
            prev,certainty = [],False
            intervals.append(i)
            valid.append(True)
        else: prev = np.array(df2[lastValid_i][0:6]).reshape(1,-2)
        pts, _ = orderCenterCoord(pts,prev)
        pts = np.array(pts)
    else: 
        jump.append(imgNumb)
        invalid+=1
        continue
    # update internal loop variables
    lastValid,invalid,lastValid_i=imgNumb,0,i
    counter+=1
    df2[i][0:6] = np.copy(pts).reshape(-1,6)
    # check if ABC is in order smaller to largest
    # if 2 consecutive trues, invert all df until interval beginning
    if not certainty:
        for [A,B,C] in pts.reshape([-1, 3, 2]):
            if np.linalg.norm(C-B)/np.linalg.norm(A-B)>(2-tol) and np.linalg.norm(C-B)>20:
                swap += 1
                if swap>5: df2[intervals[-1]:i][0:2],df2[intervals[-1]:i][4:6],swap,certainty = df2[intervals[-1]:i][4:6],df2[intervals[-1]:i][0:2],0,True
            if np.linalg.norm(A-B)/np.linalg.norm(C-B)>(2-tol) and np.linalg.norm(A-B)>20: certainty = True
    #if not i%100: print(i)
intervals.append(i)

for i in range(0,len(intervals)-1):
    if not valid[i]: continue
    beg,end = intervals[i],intervals[i+1]
    ### VERBOSE
    for j in range(beg,end):
        pts,imgNumb,img,k = df2[j][0:6].reshape(-1,2),int(df2[j][-1]),np.ones((540,960,3))*255,0
        if imgNumb in jump: continue
        for k in range(0,3):
            pt = pts.reshape(-1,2)[k]
            center = (int(np.round(pt[0]*16)),int(np.round(pt[1]*16)))
            circle(img,center,10,(255,0,0),5,shift=4)
            putText(img,str(k),(int(center[0]/16)-25, int(center[1]/16)-25),FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2) 
        imwrite('pics/cam2/'+str(imgNumb).zfill(4)+'.jpg',img)
print('Done camera #2')

Done camera #2
